In [1]:
pip install clickhouse-connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.7/788.7 kB 691.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [clickhouse-connect]lickhouse-connect]
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Импорт клиента для подключения к ClickHouse
import clickhouse_connect

# Устанавливаем соединение с ClickHouse
client = clickhouse_connect.get_client(
    host='localhost',       # Адрес сервера ClickHouse (localhost — это мой компьютер)
    port=8123,              # Порт по умолчанию для HTTP-протокола
    username='default',     # Стандартный пользователь
    password=''             # Если пароль не задан — оставляем пустым
)

# ----------- CREATE -----------
# Создаем таблицу, если она ещё не существует
def create_table():
    client.command('''
        CREATE TABLE IF NOT EXISTS test_users (
            id UInt32,        -- Уникальный ID пользователя
            name String,      -- Имя пользователя
            age UInt8         -- Возраст (до 255)
        ) ENGINE = MergeTree  -- Движок таблицы: MergeTree — основной в ClickHouse
        ORDER BY id           -- Сортировка по id для ускорения поиска
    ''')

# ----------- INSERT -----------
# Добавляем одного пользователя в таблицу
def insert_user(user_id, name, age):
    # Данные передаются в виде списка списков
    client.insert(
        'test_users',
        [[user_id, name, age]],
        column_names=['id', 'name', 'age']
    )

# ----------- READ -----------
# Получаем всех пользователей из таблицы
def get_users():
    result = client.query('SELECT * FROM test_users')  # Выполняем SQL-запрос
    return result.result_rows                          # Возвращаем строки результата, result_rows - это артибут сложного объекта result


# В ClickHouse классического UPDATE просто нет или он работает очень ограниченно. Поэтому приходится «эмулировать» обновление, чтобы добиться похожего результата.
# ----------- UPDATE (эмуляция) -----------
# Обновляем данные пользователя через удаление и повторную вставку
def update_user(user_id, new_name, new_age):
    delete_user(user_id)                               # Сначала удалим старую запись
    insert_user(user_id, new_name, new_age)            # Затем вставим новую

# ----------- DELETE -----------
# Удаляем пользователя по его id
def delete_user(user_id):
    # ClickHouse не удаляет мгновенно — фактически помечает строки
    client.command(f"ALTER TABLE test_users DELETE WHERE id = {user_id}")



In [19]:
# ----------- ТЕСТОВЫЙ ЗАПУСК -----------
# Запускается, если файл выполняется напрямую
if __name__ == '__main__':
    create_table()                                     # Создаем таблицу
    insert_user(1, 'Alice', 30)                        # Вставляем первого пользователя
    insert_user(2, 'Bob', 25)                          # Вставляем второго пользователя

    print('До обновления:', get_users())               # Читаем и печатаем

    update_user(1, 'Alicia', 31)                       # Обновляем имя и возраст пользователя 1
    print('После обновления:', get_users())

    delete_user(2)                                     # Удаляем пользователя 2
    print('После удаления:', get_users())

До обновления: [(1, 'Alicia', 31), (1, 'Alice', 30), (2, 'Bob', 25)]
После обновления: [(2, 'Bob', 25), (1, 'Alicia', 31)]
После удаления: [(2, 'Bob', 25), (1, 'Alicia', 31)]


Почему данные не исчезают сразу?
В ClickHouse ALTER TABLE ... DELETE WHERE ... — это мутирующая операция (mutation).

Она не происходит мгновенно.

ClickHouse помечает строки как удалённые, но фактическое удаление происходит в фоне и может занять время (от секунд до минут, в зависимости от нагрузки и объёма данных).

Что это значит для тебя?
Если ты сразу после delete_user запускаешь get_users(), то видишь ещё "старые" данные.

Нужно подождать, пока мутация завершится, или проверить статус мутаций.

In [17]:
def check_mutation_status(table_name='test_users', database='default'):
    # SQL-запрос для получения информации о мутациях (удалениях/обновлениях) в указанной таблице и базе данных
    query = f"""
    SELECT
        mutation_id,          -- Уникальный ID мутации
        command,              -- SQL-команда, которая была выполнена (например, DELETE)
        is_done,              -- Флаг завершения (1 — завершено, 0 — ещё выполняется)
        latest_fail_reason    -- Причина последней ошибки, если мутация не удалась
    FROM system.mutations     -- Специальная системная таблица ClickHouse со статусами мутаций
    WHERE table = '{table_name}' AND database = '{database}'
    """  # ← важно: в конце запроса не ставим точку с запятой!

    # Выполняем запрос к ClickHouse
    result = client.query(query)

    # Извлекаем строки результата в виде списка кортежей
    mutations = result.result_rows

    # Если нет активных или завершённых мутаций
    if not mutations:
        print("Мутаций для этой таблицы нет.")
        return

    # Перебираем и выводим информацию о каждой мутации
    for mutation_id, command, is_done, fail_reason in mutations:
        status = "Завершена" if is_done else "Выполняется"
        print(f"ID мутации: {mutation_id}\nКоманда: {command}\nСтатус: {status}")
        
        # Если при выполнении мутации возникла ошибка — выводим причину
        if fail_reason:
            print(f"Причина ошибки: {fail_reason}")
        
        print("---")  # Разделитель между мутациями


In [20]:
check_mutation_status()

ID мутации: mutation_3.txt
Команда: DELETE WHERE id = 1
Статус: Завершена
---
ID мутации: mutation_5.txt
Команда: DELETE WHERE id = 2
Статус: Завершена
---
ID мутации: mutation_8.txt
Команда: DELETE WHERE id = 1
Статус: Завершена
---
ID мутации: mutation_11.txt
Команда: DELETE WHERE id = 1
Статус: Завершена
---
ID мутации: mutation_13.txt
Команда: DELETE WHERE id = 2
Статус: Завершена
---
ID мутации: mutation_16.txt
Команда: DELETE WHERE id = 1
Статус: Завершена
---
ID мутации: mutation_18.txt
Команда: DELETE WHERE id = 2
Статус: Завершена
---
ID мутации: mutation_21.txt
Команда: DELETE WHERE id = 1
Статус: Завершена
---
ID мутации: mutation_23.txt
Команда: DELETE WHERE id = 2
Статус: Завершена
---
